<a href="https://colab.research.google.com/github/chenweioh/toolkits/blob/main/BEDETool_InspectionReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# User Manual for BE Inspection Report Generation

This document provides instructions for generating a Bioequivalence (BE) Study Inspection report and a detailed account document from the uploaded Excel data.

## Prerequisites

- Make sure you have the necessary Excel file containing the inspection data ready for upload.
- Your Application Number in the format "BEDE-4digit".

## Instructions

### Step 1: Run the Code

Initiate the Colab environment and run the provided Python code in its entirety.

### Step 2: Enter Application Number

When prompted, enter the Application Number using the specified format (e.g., BEDE-1234).

### Step 3: Upload Excel File

A prompt will appear asking you to upload the Excel file. Click on the prompt to select and upload your file.

### Step 4: Download Reports

After the code has completed execution, two files will be automatically downloaded:
- `BE_Inspection_Report_YYYYMMDD.xlsx`: This is the summarized inspection report.
- `BE_Inspection_Details_YYYYMMDD.docx`: This contains detailed inspection information transposed into a Word document.

## File Descriptions

- `BE_Inspection_Report_YYYYMMDD.xlsx`: Contains the inspection data with calculated fields and transformed values.
- `BE_Inspection_Details_YYYYMMDD.docx`: Provides a detailed breakdown of each inspection item in a readable format.

## Notes

- Ensure that pop-up blockers are disabled to allow downloads.
- If you encounter any errors or issues, verify the format of the input Excel file and the Application Number.

## Support

For additional help or to report issues, please contact [support@example.com](mailto:support@example.com) or reach out to the IT department.


In [3]:
!pip install python-docx  # This line installs the docx library in the Colab environment

from google.colab import files
import io
import pandas as pd
from datetime import datetime
from docx import Document
from docx.shared import Inches

# Function to parse dates and calculate the days of inspection
def calculate_days(row):
    start_date, end_date = row['Inspection dates'].split('-')
    if start_date and end_date:
        d1 = datetime.strptime(start_date.strip(), '%d/%m/%Y')
        d2 = datetime.strptime(end_date.strip(), '%d/%m/%Y')
        delta = d2 - d1
        return delta.days + 1  # including the start date
    else:
        return 0

# Function to adjust the scope of inspection
def scope_convert(scope):
    if scope == 'Clinical':
        return 'CL only'
    elif scope == 'Bioanalytical':
        return 'BA only'
    elif scope == 'Both':
        return 'CL & BA'
    else:
        return scope  # or you could return an error or a default value

# Function to determine if the inspection is study-specific
def specific_convert(type_of_inspection):
    return 'Yes' if 'specific' in type_of_inspection.lower() else 'No'

# User input for the application number
AppNo = input("Please enter the Application Number (format BEDE-4digit): ")

# Upload file
print("Please upload the Excel file.")
uploaded = files.upload()

# Get the filename of the first uploaded file
filename = next(iter(uploaded))

# Read the uploaded Excel file into a DataFrame
df = pd.read_excel(io.BytesIO(uploaded[filename]))

# Calculating days of inspection
df['Days of inspection'] = df.apply(calculate_days, axis=1)

# Dropping unnecessary column
df.drop(['Number of studies inspected, year of the study'], axis=1, inplace=True)

# Converting scope of inspection
df['CL @ BA'] = df['Scope of inspection'].apply(scope_convert)

# Adding static values
df['App. No.'] = AppNo
df['Inspection report'] = 'Available'
df['Study Specific'] = df['Type of Inspection'].apply(specific_convert)
df['Status of Compliance'] = 'Yes'

# Splitting the Inspection dates into Start and End dates
df['Date of Inspection Start'], df['Date of Inspection End'] = zip(*df['Inspection dates'].apply(lambda x: x.split('-')))

# Replace NaN values with "Not Applicable" in the entire DataFrame
df = df.fillna('Not Applicable')

# Reordering and selecting columns for the final DataFrame
df2 = df[['App. No.', 'site', 'CL @ BA', 'Inspection report', 'Regulatory Authority', 'Study Specific', 'Scope of inspection',
          'Date of Inspection Start', 'Date of Inspection End', 'Days of inspection', 'Status of Compliance', 'Number of findings',
          'Inspection conducted before/ during/ after BE Study of Interest?']]

# Saving the DataFrame to an Excel file
df2.to_excel('monitoring3.xlsx', index=False)
print('monitoring3 completed')

# Transpose DataFrame to Word document
document = Document()

df1 = df.drop(['Inspection report', 'site', 'App. No.', 'CL @ BA', 'Study Specific', 'Date of Inspection Start', 'Date of Inspection End', 'Status of Compliance'], axis=1)

for i in range(df1.shape[0]):
    records = df1.iloc[i].to_dict().items()
    table = document.add_table(rows=0, cols=2)
    table.style = 'TableGrid'
    for name, value in records:
        row_cells = table.add_row().cells
        row_cells[0].text = str(name)
        row_cells[1].text = str(value)
    document.add_page_break()

document.save('part3.docx')
print('Word document created.')

# For file download
files.download('monitoring3.xlsx')
files.download('part3.docx')

Please enter the Application Number (format BEDE-4digit): BEDE-0706
Please upload the Excel file.


Saving 3.IR.xlsx to 3.IR (1).xlsx
monitoring3 completed
Word document created.


/usr/local/lib/python3.10/dist-packages/docx/styles/styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>